In [1]:
import graphvite as gv
import graphvite.application as gap
import numpy as np
import matplotlib.pyplot as plt
# The following lines are only needed in Jupyter Notebook
from IPython.display import display, Image
%matplotlib inline
import pandas as pd

In [23]:
lib_record = pd.read_csv('Gate_2016_deidentiy_filtered.csv')
lib_record.columns = ['student_id', 'time', 'sys']
lib_record['time'] = lib_record['time'].apply(pd.to_datetime)
def filter_fun(datetime):
    if datetime.month >= 9:
        return True
    else:
        return False

lib_record['se1'] = lib_record['time'].apply(filter_fun)
lib_record_se1 = lib_record[lib_record['se1'] == True]

lib_record_se1.to_csv('lib_record_2016_se1.csv', index=False)

def filter_month_fun(datetime, month):
    if int(datetime.month) == month:
        return True
    else:
        return False

lib_record_se1['Sept'] = lib_record_se1['time'].apply(filter_month_fun, month=9)
lib_record_se1['Oct'] = lib_record_se1['time'].apply(filter_month_fun, month=10)
lib_record_se1['Nov'] = lib_record_se1['time'].apply(filter_month_fun, month=11)
lib_record_se1['Dec'] = lib_record_se1['time'].apply(filter_month_fun, month=12)


student_list = lib_record_se1['student_id'].unique()
student_dict = {}
for i in range(len(student_list)):
    student_dict[student_list[i]] = i

occ_matrix = np.array([[0]*len(student_list)]*len(student_list))

lib_record_se1_Sept = lib_record_se1[lib_record_se1['Sept'] == True]
lib_record_se1_Oct = lib_record_se1[lib_record_se1['Oct'] == True]
lib_record_se1_Nov = lib_record_se1[lib_record_se1['Nov'] == True]
lib_record_se1_Dec = lib_record_se1[lib_record_se1['Dec'] == True]

lib_record_se1_Sept.sort_values("time",inplace=True)
lib_record_se1_Oct.sort_values("time",inplace=True)
lib_record_se1_Nov.sort_values("time",inplace=True)
lib_record_se1_Dec.sort_values("time",inplace=True)

/home/wenzhy/.conda/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/wenzhy/.conda/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/wenzhy/.conda/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [6]:
lib_record_se1_Sept.head()

,student_id,time,sys,se1,Sept,Oct,Nov,Dec
966461,KENHCV2H,2016-09-01 00:21:42,25,True,True,False,False,False
189677,GRIB4HOG,2016-09-01 00:26:38,25,True,True,False,False,False
759758,MBLMMJ7G,2016-09-01 01:02:10,25,True,True,False,False,False
714027,LVXQBMWF,2016-09-01 01:28:39,25,True,True,False,False,False
929580,BYPHZL3M,2016-09-01 02:16:53,25,True,True,False,False,False


In [33]:
idx = 0
dfs = [lib_record_se1_Sept, lib_record_se1_Oct, lib_record_se1_Nov, lib_record_se1_Dec]
for df in dfs:    
    df['first_week']  = df['time'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
    df['second_week'] = df['time'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
    df['third_week']  = df['time'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
    df['forth_week']  = df['time'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

    df_first_week  = df[df['first_week'] == 1]
    df_second_week = df[df['second_week'] == 1]
    df_third_week  = df[df['third_week'] == 1]
    df_forth_week  = df[df['forth_week'] == 1] 

    for demo in [df_first_week, df_second_week, df_third_week, df_forth_week]:
        occ_matrix = np.array([[0]*len(student_list)]*len(student_list))
        demo_list = []
        for r in demo.iterrows():
            demo_list.append((r[1]['student_id'], r[1]['time']))

        time_delta = pd.Timedelta(30, unit='s')
        for i in range(len(demo_list)):
            oob = False
            j = i + 1
            while ((oob == False) and j < len(demo_list)):
                if demo_list[j][1] < demo_list[i][1] + time_delta:
                    # print(demo_list[i][0], demo_list[j][0])
                    occ_matrix[student_dict[demo_list[i][0]]][student_dict[demo_list[j][0]]] += 1
                    occ_matrix[student_dict[demo_list[j][0]]][student_dict[demo_list[i][0]]] += 1
                    j += 1
                else:
                    oob = True
        idx += 1
        # np.save('lib_co_occ_week_'+ str(idx) +'.npy', occ_matrix)
        edge_list = []
        for i in range(len(student_list)):
            for j in range(len(student_list)):
                if i < j:
                    edge_list.append((i, j, occ_matrix[i][j]))

        cnt = 0
        with open('lib_co_occ_se1_week_'+ str(idx) +'_30_sec_intervals.edgelist', 'w') as f:
            for i in edge_list:
                if i[2] >= 2:
                    cnt += 1
                    f.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')
        print(cnt)

19978
35488
36469
44344
44422
37041
40669
55116
40236
42127
50608
42064
23459
16638
7119
267


In [5]:
lib_record = pd.read_csv('Gate_2016_deidentiy_filtered.csv')
lib_record.columns = ['student_id', 'time', 'sys']
lib_record['time'] = lib_record['time'].apply(pd.to_datetime)
def filter_fun(datetime):
    if datetime.month >= 9:
        return True
    else:
        return False

lib_record['se1'] = lib_record['time'].apply(filter_fun)
lib_record_se1 = lib_record[lib_record['se1'] == True]
student_list = lib_record_se1['student_id'].unique()
student_dict = {}
for i in range(len(student_list)):
    student_dict[student_list[i]] = i
len(student_list)

15503

In [17]:
Threshold = 2

for cnt in range(1, 15):
    Filename = 'lib_co_occ_se1_week_'+str(cnt)+'_30_sec_intervals.edgelist'
    fr = open('lib_co_occ_se1_week_'+str(cnt)+'_'+ str(Threshold)+'.edgelist', 'w')
    with open(Filename, 'r') as f:
        for line in f.readlines():
            if int(line.split()[2]) >= Threshold:
                fr.write(line)
    fr.close()    

In [18]:
import traceback

student_features = pd.read_csv('2016_se1_lib_lms.csv')
student_features.head()
labels = student_features[['MASKED_STUDENT_ID', 'label_atRist']]      

def get_vec_embed(filename):
        app = gap.GraphApplication(dim=64)
        app.load(file_name=filename)
        app.build()
        app.train(shuffle_base=1,augmentation_step=30)
        vertex_embeddings = app.solver.vertex_embeddings
        return vertex_embeddings
        
for cnt in range(1, 15):
    print(cnt)
    Filename = 'lib_co_occ_se1_week_'+str(cnt)+'_'+ str(Threshold)+'.edgelist'


    nodeset = []
    with open(Filename, 'r') as f:
        for line in f.readlines():
            nodeset.append(line.split()[0])
            nodeset.append(line.split()[1])
    node_list = list(set(nodeset))
    node_list = sorted(node_list)
    try:
        vertex_embeddings = get_vec_embed(Filename)
    except:
        print(traceback.print_exc())
    nodevec_dict = {}
    for i in range(len(node_list)):
        nodevec_dict[node_list[i]] = vertex_embeddings[i]

    def merge_vec(MASKED_STUDENT_ID):
        try:
            # print(student_dict[MASKED_STUDENT_ID])
            return nodevec_dict[str(student_dict[MASKED_STUDENT_ID])]
        except:
            return 0
    labels['week_' + str(cnt)] = labels['MASKED_STUDENT_ID'].apply(merge_vec)
# labels.to_csv(Filename +'_embeddings.csv', index=False)

1
[time] GraphApplication.load: 0.0103838 s
[time] GraphApplication.build: 0.295786 s
[time] GraphApplication.train: 1.53457 s
2
[time] GraphApplication.load: 0.0171716 s


/home/wenzhy/.conda/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[time] GraphApplication.build: 0.247726 s
[time] GraphApplication.train: 2.15595 s
3
[time] GraphApplication.load: 0.0193338 s
[time] GraphApplication.build: 0.252641 s
[time] GraphApplication.train: 1.9944 s
4
[time] GraphApplication.load: 0.0205562 s
[time] GraphApplication.build: 0.247499 s
[time] GraphApplication.train: 2.46893 s
5
[time] GraphApplication.load: 0.020483 s
[time] GraphApplication.build: 0.251912 s
[time] GraphApplication.train: 2.57962 s
6
[time] GraphApplication.load: 0.0176895 s
[time] GraphApplication.build: 0.254514 s
[time] GraphApplication.train: 2.18519 s
7
[time] GraphApplication.load: 0.01893 s
[time] GraphApplication.build: 0.248878 s
[time] GraphApplication.train: 2.5709 s
8
[time] GraphApplication.load: 0.0253959 s
[time] GraphApplication.build: 0.258452 s
[time] GraphApplication.train: 2.77952 s
9
[time] GraphApplication.load: 0.0196049 s
[time] GraphApplication.build: 0.257222 s
[time] GraphApplication.train: 2.54925 s
10
[time] GraphApplication.load: 

In [20]:
labels.to_csv('se1_weekly_node_embeddings_30_sec_2s.csv')

In [19]:
labels.head()

,MASKED_STUDENT_ID,label_atRist,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14
0,8TMIKVZ5,0,"[9e-44, 0.0, 6e-45, 0.0, 7.4932477e-25, 3.0893...","[9e-44, 0.0, 6e-45, 0.0, 7.4932477e-25, 3.0893...","[0.0, 0.0, 1.36e-43, 0.0, 7.1454784e-25, 3.089...","[8.012797e-23, 3.0893e-41, 6.128826e-25, 3.089...","[8.012797e-23, 3.0893e-41, 6.128826e-25, 3.089...","[8.012797e-23, 3.0893e-41, 6.128826e-25, 3.089...",0,"[7.2820754e-09, 4.5622e-41, 7.282111e-09, 4.56...","[4.5e-44, 0.0, 4.5e-44, 0.0, 9e-44, 0.0, 6e-45...","[2.303229e-23, 3.0893e-41, -5.14843e-27, 4.562...",0,0,0,0
1,N2YYKTMZ,0,"[2.0, 1.107e-42, 2.0, 3.755e-42, 2.0, 2.052e-4...","[2.0, 1.107e-42, 2.0, 3.755e-42, 2.0, 2.052e-4...","[8.772042e-37, 9.858019e+18, 4.6e-44, 0.0, 6.5...","[3.81e-43, 0.0, 7.4e-43, 0.0, 2.0, 5.52e-43, 4...","[3.81e-43, 0.0, 7.4e-43, 0.0, 2.0, 5.52e-43, 4...","[3.81e-43, 0.0, 7.4e-43, 0.0, 2.0, 5.52e-43, 4...","[1.35e-43, 0.0, 9e-44, 0.0, 9e-44, 0.0, 6e-45,...","[7.1519794e-29, 3.0893e-41, 7.1519794e-29, 3.0...","[1.35e-43, 0.0, 4.5e-44, 0.0, 9e-44, 0.0, 6e-4...","[-0.7575744, -0.44993606, 1.0244414, -0.394497...","[2.303229e-23, 3.0893e-41, -5.14843e-27, 4.562...","[2.303229e-23, 3.0893e-41, -5.14843e-27, 4.562...",0,"[0.042909198, 0.023664782, -0.049640283, 0.029..."
2,BZRW4GD3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HJTBF62Q,0,0,"[5.7232056e-25, 3.0893e-41, 5.7239156e-25, 3.0...",0,0,"[7.1519794e-29, 3.0893e-41, 7.1519794e-29, 3.0...",0,"[1.35e-43, 0.0, 9e-44, 0.0, 9e-44, 0.0, 6e-45,...","[6.393229e-09, 4.5622e-41, 6.3932006e-09, 4.56...","[7.1519794e-29, 3.0893e-41, 1969.0059, 4.5622e...","[-0.6560756, -0.38873905, 0.87246454, -0.33593...",0,"[-0.7371482, -0.4444602, 0.9926179, -0.3789609...","[-0.037751608, -0.042469725, -0.057543498, 0.0...","[0.04691473, 0.028493699, -0.06408096, 0.02720..."
4,33XUIDIG,0,0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,0,"[1.35e-43, 0.0, 4.5e-44, 0.0, 9e-44, 0.0, 6e-4...","[-0.6840235, -0.41992414, 0.922771, -0.3539974...",0,0,0,"[0.043734755, 0.021909606, -0.05502832, 0.0218..."


In [16]:
float('3.0646e-41')

3.0646e-41

In [1]:
import graphvite as gv
import graphvite.application as gap
import numpy as np
import matplotlib.pyplot as plt
# The following lines are only needed in Jupyter Notebook
# from IPython.display import display, Image
# %matplotlib inline
import pandas as pd

def get_vec_embed(filename):
    app = gap.GraphApplication(dim=32)
    app.load(file_name=filename)
    app.build(batch_size=100)
    app.train(shuffle_base=4)
    vertex_embeddings = app.solver.vertex_embeddings
    return vertex_embeddings

Filename = 'lib_co_occ_se1_week_1.edgelist'
vertex_embeddings = get_vec_embed(Filename)

[time] GraphApplication.load: 0.0083003 s
[time] GraphApplication.build: 0.470265 s
[time] GraphApplication.train: 34.1616 s
